In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
q = pd.read_csv("Questions.csv", encoding='ISO-8859-1')
t = pd.read_csv("Tags.csv", encoding='ISO-8859-1')
q = q[:1000]

In [3]:
q.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
t.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [5]:
q = q[["Id","Title","Body"]]
q.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [6]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    return re.sub(cleanr, '', raw_html)

In [7]:
q['Body'] = q['Body'].apply(cleanhtml)
q["Title"] = q["Title"].map(str) + " " + q["Body"]
q = q[["Id","Title"]]
q["Title"] = q["Title"].apply(lambda x: re.sub('[^0-9a-zA-Z ]+', '', x.lower()))
q.head()

,Id,Title
0,80,sqlstatementexecute multiple queries in one s...
1,90,good branching and merging tutorials for torto...
2,120,aspnet site maps has anyone got experience cre...
3,180,function for creating color wheels this is som...
4,260,adding scripting functionality to net applicat...


In [8]:
def getKeys(index):
    tags = t[t["Id"] == index]
    myTags = list(tags["Tag"])
    out = ""
    #for x in myTags:
    #    out+= str(x)+","
    #return out[0:len(out)-1]
    return myTags[0]

In [9]:
q["Tag"] = q['Id'].apply(getKeys)
q = q.set_index('Id')
q.head()

,Title,Tag
Id,,
80,sqlstatementexecute multiple queries in one s...,flex
90,good branching and merging tutorials for torto...,svn
120,aspnet site maps has anyone got experience cre...,sql
180,function for creating color wheels this is som...,algorithm
260,adding scripting functionality to net applicat...,c#


In [11]:
q.to_csv("cleantitletag.csv")